### Example: quantum circuits

*This example is adapted from a blogpost by Carlo Lepelaars and originally appeared on his [personal website](https://carlo.ai/posts/fastcore-quantum). It is reposted here with permission.*

In [ ]:
import numpy as np
from numpy.testing import assert_almost_equal
from fasttransform import Transform, Pipeline

## Transform

Transform is a fundamental building block to manipulate data in Python. While extremely simple to use, it is also flexible. Transforms can be set up to change behavior based on the input type, also called type dispatch. Transforms can also be made reversible. Keep this reversibility in mind for the quantum part later! Here is a simple example of a Transform that can square a number and take square root as reverse:

In [ ]:
class S(Transform):
    def encodes(self, x): return x ** 2
    def decodes(self, x): return x ** 0.5
            
assert S()(10) == 100
assert S().decode(100) == 10
assert S().decode(S()(10)) == 10

A Transform with only `encode` can be defined even simpler with a `lambda` function. By default, `decode` returns its input (i.e. does nothing / "no-op"):

In [ ]:
square = Transform(lambda x: x ** 2)
assert square(10) == 100
assert square.decode(100) == 100

`Transform` can also be used as a decorator to turn a function into a `Transform`:

In [ ]:
@Transform
def square(x): return x ** 2
square(10) # 100
type(square) # <class 'fastcore.transform.Transform'>

fasttransform.transform.Transform

A powerful feature of `Transform` is type dispatch:

In [ ]:
class MultiS(Transform):
    def encodes(self, x: int | float | complex | tuple): return x**2
    def encodes(self, x: list): return [x**2 for x in x]
    def decodes(self, x: int | float | complex | tuple): return x**0.5
    def decodes(self, x: list): return [x**0.5 for x in x]

ms = MultiS()
ms

MultiS(enc:2,dec:2)

In [ ]:
# Lists
# By default, Transform processes lists as a whole
# 2nd encodes method is called
assert ms([1, 2, 3]) == [1, 4, 9]

In [ ]:
# 2nd decodes method is called
assert ms.decode([1, 4, 9]) == [1.0, 2.0, 3.0]                     

In [ ]:
# Tuples
# By default, Transform processes tuples elementwise
# 1st decodes method is called
assert ms((1, 2, 3)) == (1, 4, 9)

In [ ]:
# 1st decodes method is called
assert ms.decode((1, 4, 9)) == (1.0, 2.0, 3.0)

In [ ]:
# Complex numbers
# 1st encodes method is called on complex number
assert ms(10.0j) == (-100+0j)

In [ ]:
# 1st decodes method is called on complex number
assert ms.decode(ms(10.0j)) == (6.123233995736766e-16+10j)

`Transform` automatically routes input to the corresponding method based on the input type. `Transform` will operate on lists as a whole, while tuples are processed elementwise. Processing lists and arrays as one object is a common use case in data science, for example if we are processing batches of images (3D arrays) for machine learning. If you want a `Transform` that also transforms tuples as a whole, use `ItemTransform`. To support inplace transforms use `InplaceTransform`.

## Pipeline

Now that we have a firm understanding of `Transform`, `Pipeline` can be used to chain them together:

In [ ]:
class S(Transform):
    "Square a number. Reverse is square root."
    def encodes(self, x): return x ** 2
    def decodes(self, x): return x ** 0.5
    
class A(Transform):
    "Add 1. Reverse is subtract 1."
    def encodes(self, x): return x + 1
    def decodes(self, x): return x - 1

pipe = Pipeline([S(), A()])
assert pipe(10) == 101 # 10**2 + 1 = 101
assert pipe.decode(10) == 3.0 # (10 - 1)**0.5 = 3.0
assert pipe.decode(pipe(10)) == 10 # (10**2 + 1 - 1)**0.5 = 10

I hope you appreciate the simplicity of `Transform` and `Pipeline`. This year I've been working on an open-source library for quantum computing that connects several quantum computing frameworks and standards (For example, [Qiskit](https://github.com/Qiskit/qiskit), [PennyLane](https://github.com/PennyLaneAI/pennylane) and [OpenQASM](https://en.wikipedia.org/wiki/OpenQASM). At the base of it lies [numpy](https://github.com/numpy/numpy). To generalize quantum circuits I tried using [scikit-learn's Pipeline](https://scikit-learn.org/stable/modules/compose.html) functionality. Unfortunately this led to a lot of boilerplate code and unnecessary features. fastcore offers a more elegant and promising solution for the use case of constructing quantum circuits, which we will explore in the next section.

## Quantum

Quantum computing processes can be simulated on classical computers by transforming a [statevector](https://en.wikipedia.org/wiki/Quantum_state) (i.e. list of [complex numbers](https://en.wikipedia.org/wiki/Complex_number)) through a series of [reversible quantum logic gates](https://en.wikipedia.org/wiki/Quantum_logic_gate) (i.e. matrix of complex numbers). The state and gates are subject to constraints, but the basic operation is (reversible) vector-matrix multiplication. This is a perfect use case for fastcore's `Transform` and `Pipeline`. To illustrate this point we will start with manipulating a single qubit using `Transform`.

### Single Qubit

A statevector contains 2 complex numbers which gives us the likelihood of obtaining a $0$ or $1$. This is called a [superposition](https://en.wikipedia.org/wiki/Quantum_superposition) between $0$ and $1$. The numbers in the vector are called [probability amplitudes](https://www.youtube.com/watch?v=XrKl38ZVofo) and can be converted to probabilities by computing $|x|^2$ (i.e. the absolute value squared), where $x$ is the statevector. Probabilities must always sum to $1$, so a valid qubit state $[\alpha, \beta]$ must have $|\alpha|^2 + |\beta|^2 = 1$.

A shortcut used for writing quantum states is [Dirac notation](https://en.wikipedia.org/wiki/Bra%E2%80%93ket_notation). For example $|0\rangle=[1, 0]^T$ (i.e. always 0) and $|1\rangle=[0, 1]^T$ (i.e. always 1). Other valid single qubit states include $\frac{1}{\sqrt{2}}(|0\rangle + |1\rangle) = \begin{bmatrix} \frac{1}{\sqrt{2}} \frac{1}{\sqrt{2}} \end{bmatrix}^T$ for a perfectly equal superposition and $\frac{1+i}{2}|0\rangle + \frac{1-i}{2}|1\rangle = \begin{bmatrix} \frac{1+i}{2} \frac{1-i}{2} \end{bmatrix}^T$ for a superposition of both the real and complex parts.

Using `Transform` we can easily define quantum logic gates. We define a base `Transform` that can do vector-matrix multiplication in a reversible way and implement common quantum gates. The [I (identity) gate](https://en.wikipedia.org/wiki/Quantum_logic_gate#:~:text=in%20the%20literature.-,Identity%20gate,-%5Bedit%5D) does nothing, the [X (NOT) gate](https://en.wikipedia.org/wiki/Quantum_logic_gate#:~:text=multi%2Dqubit%20circuits.-,Pauli%20gates%20(X%2CY%2CZ),-%5Bedit%5D) flips the qubit from $|0\rangle$ to $|1\rangle$ and vice versa. The [Hadamard gate](https://en.wikipedia.org/wiki/Quantum_logic_gate#:~:text=%5B16%5D-,Hadamard%20gate,-%5Bedit%5D) turns a qubit into superposition (i.e. turn $|0\rangle$ into $\frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$).

In [ ]:
class _Q(Transform):
    "Base transform for quantum gates"
    def encodes(self, x): return x @ self.gate
    def decodes(self, x): return x @ self.gate.conj().T
                     
class I(_Q):
    "Identity gate. Does nothing."
    gate = np.array([[1, 0], 
                     [0, 1]])
    
class X(_Q):
    "X (NOT) gate. Flips from |0> to |1> and vice versa."
    gate = np.array([[0, 1], 
                     [1, 0]])
    
class H(_Q):
    "Hadamard (Superposition) gate. Turns a qubit into a superposition."
    gate = np.array([[1, 1], 
                     [1, -1]]) / np.sqrt(2)

This allows us to easily play with quantum gates:

In [ ]:
zero_state = [1+0j, 0+0j] # Basis state |0>
superposition_state = np.array([0.5+0.5j, 0.5-0.5j])  # Complex superposition

In [ ]:
# Identity operation
i = I() 
assert_almost_equal(i(zero_state), np.array([1.+0.j, 0.+0.j])) # (|0>)
assert_almost_equal(i.decode(zero_state), np.array([1.+0.j, 0.+0.j]))  # (|0>)                     

In [ ]:
# X (NOT) operation 
x = X()
assert_almost_equal(x(zero_state), np.array([0+0j, 1+0j]))  # (|1>)
assert_almost_equal(x.decode(x(zero_state)), np.array([1.+0.j, 0.+0.j]))  # (|0>)
assert_almost_equal(x(superposition_state), np.array([0.5-0.5j, 0.5+0.5j]))  # (flips sign of complex part)

In [ ]:
# Hadamard gate tests
h = H()
assert_almost_equal(h(zero_state), np.array([0.707+0.j, 0.707+0.j]), decimal=3)  # (superposition)
assert_almost_equal(h(superposition_state), np.array([0.707+0.j, 0.+0.707j]), decimal=3)  # (phase state)
assert_almost_equal(h.decode(h(superposition_state)), np.array([0.5+0.5j, 0.5-0.5j]))  # (complex superposition)

I hope this gives you an appreciation for the simplicity of `Transform` and why it works for quantum, even if the details and meaning of these operations might not be clear yet. If you are interested in learning about quantum computing I highly recommend reading the amazing educational material on [quantum.country](https://quantum.country). A great quantum textbook is ["Quantum Computing and Quantum Information" by Michael Nielsen and Isaac Chuang](https://www.amazon.com/Quantum-Computation-Information-10th-Anniversary/dp/1107002176?crid=VJB5339SIYMB&dib=eyJ2IjoiMSJ9.eSzRaEDPfZNRNJrJx5nhCTp0oQkLWZkujqGOiuevj_sHJbKkL1Gzm-lj8Yg2C3VA11v61b8iZ-lMx5ju_Os5gYarrMvkrmlT5axEqzNql0rF7vdjvVx1wdd4zI7PnuATJf-wu3J6ifpngHnkYPoPII5TV6ywBOK4K34NY0JAHOkhAfyndoZWV3oxb3asowdT77R7mV6CA0CQUD6Vv2zmMFjeeGxBJO4Sexl8HlJNLRQ.fjinpRoMwoKMQ9jlWLL8j1bgA5KWKM-F8lPFzA-5hvo&dib_tag=se&keywords=quantum+computation+and+information&qid=1732985753&sprefix=quantum+computation+and+information,aps,213&sr=8-1&linkCode=sl1&tag=carloai-20&linkId=0a6f532c12be8d87a7717d00f3c91991&language=en_US&ref_=as_li_ss_tl).

Another essential component of quantum is [measurement](https://en.wikipedia.org/wiki/Measurement_in_quantum_mechanics). This transforms the quantum state into a probability distribution we can sample from. Note that these operations are not reversible. This is because after measurement, a quantum state [collapses](https://en.wikipedia.org/wiki/Wave_function_collapse):

In [ ]:
class M(Transform):
    "Turn a quantum statevector into a probability distribution"
    def encodes(self, x): return np.abs(x)**2
    def decodes(self, x): return NotImplementedError("No inverse exists for absolute value.")

class Samp(Transform):
    "Sample from a probability distribution"
    def encodes(self, x): return format(np.random.choice(len(x), p=x), f'0{int(np.log2(len(x)))}b')
    def decodes(self, x): return NotImplementedError("Sampling is not reversible.")

m = M()
samp = Samp()

In [ ]:
# Sampling from zero state (|0>)
zero_state = [1+0j, 0+0j]
mzs = m(zero_state) # Transforms [1+0j, 0+0j] -> [1, 0]

assert_almost_equal(mzs, np.array([1+0j,0+0j]))                     
assert samp(mzs) == '0'

In [ ]:
# Sampling from equal superposition
equal_superposition = [0.707, 0.707]
mes = m(equal_superposition) # Transforms [0.707, 0.707] -> [0.5, 0.5] (A coin flip. i.e. Bernoulli distribution))

In [ ]:
mes = mes / mes.sum()  # not in original blog but needed as otherwises numers dont add to 1 because of rounding
assert_almost_equal(mes, np.array([0.5,0.5]))
assert samp(mes) in '01' # 0 or 1 with equal probability

In [ ]:
# Sampling from complex superposition
complex_superposition = [0.5+0.5j, 0.5-0.5j]
mcs = m(complex_superposition) # Transforms [0.5+0.5j, 0.5-0.5j] -> [0.5, 0.5] (A coin flip. i.e. Bernoulli distribution)
assert_almost_equal(mcs, np.array([0.5,0.5]))

In [ ]:
assert samp(mcs) in '01'  # Result is 0 or 1 with equal probability

We can now build a full quantum circuit using `Pipeline`. Note that a quantum pipeline is only reversible if it does not include measurement or sampling:

In [ ]:
qc = Pipeline([X(), H(), I(), M(), Samp()])
# X transforms [1, 0] -> [0, 1]
# H transforms [0, 1] -> [0.707+0j, -0.707+0j]
# I transforms [0.707+0j, -0.707+0j] -> [0.707+0j, -0.707+0j]
# M transforms [0.707+0j, -0.707+0j] -> [0.5, 0.5]
# Samp samples from random distribution [0.5, 0.5]
assert qc(zero_state) in '01' # 0 or 1 with equal probability

### Multi Qubit

Even though we only discussed single qubit cases we can the potential of `Transform` for quantum. It gets even more powerful if we start working with multiple qubits. The representation of a quantum state on a classical computer grows exponentially with each qubit, because each qubit can be [entangled](https://en.wikipedia.org/wiki/Quantum_entanglement) with others. We therefore need a matrix of $2^n$ x $2^n$ to represent a transformation of $n$ qubits. The statevector for $n$ qubits contains $2^n$ complex numbers. Single qubit gates can be combined through the [Tensor (Kronecker) product](https://www.math3ma.com/blog/the-tensor-product-demystified), which we handle in `Concat`:

In [ ]:
class Concat(Transform):
    "Combine single qubit gates into a multi-qubit gate"
    def __init__(self, gates): self.gates = gates
    # Concatenate 2 or more gates
    def encodes(self, x): return x @ np.kron(*[g.gate for g in self.gates])
    # Reverse propagation for all gates
    def decodes(self, x):
        for g in reversed(self.gates): x = x @ np.kron(g.gate.conj().T, np.eye(len(x) // g.gate.shape[0]))
        return x

By concatenating single qubit gates we can construct multi-qubit circuits, while keeping the code extremely simple. However, some gates are fundamentally multi-qubit and cannot be constructed from single qubits. One example is the [Controlled NOT (CNOT) gate](https://en.wikipedia.org/wiki/Controlled_NOT_gate), which flips the 2nd qubit from $|0\rangle$ to $|1\rangle$ based on the value of the 1st qubit. When we combine Hadamard on the 1st qubit and a CNOT gate we get the well known [Bell state](https://en.wikipedia.org/wiki/Bell_state). This is a classic example of fully entangling qubits where we obtain $00$ or $11$ with equal probability.

In [ ]:
class CNOT(_Q):
    "Controlled NOT gate"
    def __init__(self): self.gate = np.array([[1, 0, 0, 0], 
                                              [0, 1, 0, 0], 
                                              [0, 0, 0, 1], 
                                              [0, 0, 1, 0]])
                     
two_qubit_zero_state = np.array([1+0j, 0+0j, 0+0j, 0+0j]) # |00>
qc = Pipeline([Concat([H(), I()]), CNOT(), M(), Samp()])
# Concat([H(), I()]) transforms [1, 0, 0, 0] -> [0.707, 0, 0.707, 0]
# CNOT() transforms [0.707, 0, 0.707, 0] -> [0.707, 0, 0, 0.707]
# M() transforms [0.707, 0, 0, 0.707] -> [0.5, 0, 0, 0.5]
# Samp() samples from [0.5, 0, 0, 0.5] (50% chance at 00 and 50% chance at 3, which is 11 in binary)
assert qc(two_qubit_zero_state) in ('00','11') # 00 or 11 with equal probability

These techniques can be used to simulate and analyze more complicated multi-qubit circuits. The nice thing about building quantum circuits like this is that we can analyze every step and get a good understanding of what is happening. It also allows us to precisely explore techniques like [quantum error correction](https://en.wikipedia.org/wiki/Quantum_error_correction). However, for large scale quantum circuits the matrices are huge and [real quantum computers](https://en.wikipedia.org/wiki/Quantum_computing) are needed to do the computation. Real quantum computers directly leverage properties of quantum mechanics like entanglement, superposition and interference. These are things that a classical computer can simulate, but cannot natively perform like a real quantum computer. Exploiting quantum properties can result in [potentially exponential speedups](https://en.wikipedia.org/wiki/Shor's_algorithm). For more information on where quantum computers excel check out [Ronald de Wolf's great paper on "The Potential Impact of Quantum Computers on Society (2017)](https://arxiv.org/abs/1712.05380). This paper has stood the test of time even though new breakthroughs have been achieved.m

## Closing

If you made it all the way to the end, congratulations! I salute you! 🫡 Having an understanding of both advanced Python and quantum computing is a rare skillset. If you are interested in this intersection you might be interested in exploring [Quantum Machine Learning](https://en.wikipedia.org/wiki/Quantum_machine_learning). This field combines the optimization we often see in data science and machine learning with quantum computing. If this piques your interest, one of the best textbooks around is ["Machine Learning with Quantum Computers by Maria Schuld and Francesco Petruccione"](https://www.amazon.com/Machine-Learning-with-Quantum-Computers-_Quantum-Science-and-Technology_/dp/3030830977?crid=19KO4XCS8WOP8&dib=eyJ2IjoiMSJ9.dNxHwOOX1eQ--tz1drbE8Q5FSOExFYW2vGdIzGTQPD1F1JdSLXDUmGz1gHgNpNrZqLfVexuwQnMc9n8vbR6sNfcs0ijwVKQdLeinveYC5VvKXkCidBuUYhL7U-ftyItGwdtN01Xrz5pC23LQWBhguzaYYMqQ6rSRNHdd8A5h8b-Ly-EWMBOffxQ78eOy6Xs24CERAcdYF9k-UTxOR9fvOZ4mq-_rHleRj9waom7vcRM.ZGWt483eALBI0Qe-jOtxRV0E1C5-SMyDTOhz_QBiZfA&dib_tag=se&keywords=machine+learning+quantum+computers&qid=1732988249&s=books&sprefix=machine+learning+quantum+computer,stripbooks-intl-ship,194&sr=1-1&linkCode=sl1&tag=carloai-20&linkId=ca70ef7f87b0a468dada46c96681d181&language=en_US&ref_=as_li_ss_tl).

## Learning resources

If you are interested in learning more about quantum, I recommend the following resources:

### Online Resources[](https://carlo.ai/posts/fastcore-quantum#online-resources)

- [Quantum Country - Quantum Education Essays](https://quantum.country/)
- [IBM Quantum Learning Platform](https://learning.quantum.ibm.com/)
- [PennyLane Quantum Learning Platform](https://pennylane.ai/qml)
- [q4p - Quantum Computing for (Python) Programmers (by Carlo Lepelaars)](https://github.com/CarloLepelaars/q4p)
- [Paper: The Potential Impact of Quantum Computers on Society (2017) by Ronald de Wolf](https://arxiv.org/abs/1712.05380)

### Books[](https://carlo.ai/posts/fastcore-quantum#books)

- [Quantum Computation and Quantum Information (Nielsen & Chuang)](https://www.amazon.com/Quantum-Computation-Information-10th-Anniversary/dp/1107002176?crid=VJB5339SIYMB&dib=eyJ2IjoiMSJ9.eSzRaEDPfZNRNJrJx5nhCTp0oQkLWZkujqGOiuevj_sHJbKkL1Gzm-lj8Yg2C3VA11v61b8iZ-lMx5ju_Os5gYarrMvkrmlT5axEqzNql0rF7vdjvVx1wdd4zI7PnuATJf-wu3J6ifpngHnkYPoPII5TV6ywBOK4K34NY0JAHOkhAfyndoZWV3oxb3asowdT77R7mV6CA0CQUD6Vv2zmMFjeeGxBJO4Sexl8HlJNLRQ.fjinpRoMwoKMQ9jlWLL8j1bgA5KWKM-F8lPFzA-5hvo&dib_tag=se&keywords=quantum+computation+and+information&qid=1732985753&sprefix=quantum+computation+and+information,aps,213&sr=8-1&linkCode=sl1&tag=carloai-20&linkId=0a6f532c12be8d87a7717d00f3c91991&language=en_US&ref_=as_li_ss_tl)
- [Machine Learning with Quantum Computers (Schuld & Petruccione)](https://www.amazon.com/Machine-Learning-with-Quantum-Computers-_Quantum-Science-and-Technology_/dp/3030830977?crid=19KO4XCS8WOP8&dib=eyJ2IjoiMSJ9.dNxHwOOX1eQ--tz1drbE8Q5FSOExFYW2vGdIzGTQPD1F1JdSLXDUmGz1gHgNpNrZqLfVexuwQnMc9n8vbR6sNfcs0ijwVKQdLeinveYC5VvKXkCidBuUYhL7U-ftyItGwdtN01Xrz5pC23LQWBhguzaYYMqQ6rSRNHdd8A5h8b-Ly-EWMBOffxQ78eOy6Xs24CERAcdYF9k-UTxOR9fvOZ4mq-_rHleRj9waom7vcRM.ZGWt483eALBI0Qe-jOtxRV0E1C5-SMyDTOhz_QBiZfA&dib_tag=se&keywords=machine+learning+quantum+computers&qid=1732988249&s=books&sprefix=machine+learning+quantum+computer,stripbooks-intl-ship,194&sr=1-1&linkCode=sl1&tag=carloai-20&linkId=ca70ef7f87b0a468dada46c96681d181&language=en_US&ref_=as_li_ss_tl)
- [Quantum Mechanics: The Theoretical Minimum (Susskind)](https://www.amazon.com/Quantum-Mechanics-Theoretical-Leonard-Susskind/dp/0465062903?_encoding=UTF8&dib_tag=se&dib=eyJ2IjoiMSJ9.bQHxFT9WpIkYRkeCumwpdFVbCjwd5MEHWzWJBHklnGv2UtBa5W-NMv4XeVzQxQBDAskRXb-pN1xeDbczsDL199x-RfGYfKBIXxM2v-2MY25Bk6ZANGLEcIWY2oYafqLc.YEqLRNNMX_gWfLDcAs3fgkEw-13sRkvz5yjdff_tDLI&qid=1732988786&sr=1-3&linkCode=sl1&tag=carloai-20&linkId=109f846df78407b69dccf53cefbf8427&language=en_US&ref_=as_li_ss_tl)
- [Modern Quantum Mechanics (Sakurai)](https://www.amazon.com/Modern-Quantum-Mechanics-J-Sakurai/dp/1108473229?_encoding=UTF8&dib_tag=se&dib=eyJ2IjoiMSJ9.9aiac9T0bD5vFns_B8FoL4VsZn01tfjZm7mj2xOAZQ1W2pM7wyq6nfygNTzWezDFlKiPhAmdLbfyqqOEiZcwK3DZmhx27oLxfQ8Xhruwy7mTk0CoCb1JbZgORWCyRQBJA9AoGY5yB3Da8DtnydBoz-ywC-eDJ29NqklfQS1DIoiJv6b39vKb-GoOMKmFyy8A6iR20tM8KTQwGTkKFYsLu-ZW4twj99IYD0uJPNRbVgM.TLySH0uJr2O7SoAxzEP_SdWdhE44gnkv5BcFdk7cU74&qid=1732995963&sr=1-1&linkCode=sl1&tag=carloai-20&linkId=8aaf375bbfc7b9f162774e1693243980&language=en_US&ref_=as_li_ss_tl)

### YouTube[](https://carlo.ai/posts/fastcore-quantum#youtube)

- [Playlist - Introduction to Quantum information Science (Artur Ekert)](https://www.youtube.com/@ArturEkert/playlists)
- [Playlist - Quantum Machine Learning (Peter Wittek)](https://www.youtube.com/watch?v=QtWCmO_KIlg&list=PLmRxgFnCIhaMgvot-Xuym_hn69lmzIokg)
- [Playlist - Quantum Paradoxes (Maria Violaris)](https://www.youtube.com/watch?v=Pz829XZIxXg&list=PLOFEBzvs-VvoQP-EVyd5Di3UrPPc2YKIc&index=1)
- [Playlist - The History of Quantum Computing (Interviews)](https://www.youtube.com/watch?v=6qD9XElTpCE&list=PLOFEBzvs-VvpNNq221OCZzCZjHadM2NHR&index=1)
- [Playlist - Maths of Quantum Mechanics](https://www.youtube.com/watch?v=3nvbBEzfmE8&list=PL8ER5-vAoiHAWm1UcZsiauUGPlJChgNXC)
- [Channel - Looking Glass Universe](https://www.youtube.com/@LookingGlassUniverse)
- [Video - The Map of Quantum Computing](https://www.youtube.com/watch?v=-UlxHPIEVqA)
- [Video - Logic Gates Rotate Qubits (Josh's Channel)](https://www.youtube.com/watch?v=ZBaXPY_0TNI)
- [Video - How Quantum Entanglement Works](https://www.youtube.com/watch?v=nxIGNbcVKtA)
- [Video - Interpretations of Quantum Mechanics](https://www.youtube.com/watch?v=mqofuYCz9gs)